### Offtarget Analysis of CjCas Editors
Summarize replicates and substract control values.

In [35]:
import pandas as pd
import seaborn as sn
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gaussian_kde
import scipy.stats as stats
from Bio.Seq import Seq
from Bio.SeqUtils import MeltingTemp as mt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from bioinfokit.analys import stat
import scikit_posthocs as sp
from Bio.SeqUtils import GC
import numpy as np
from collections import Counter, OrderedDict
from os import listdir

In [36]:
lsize=14
titlesize=16
axesthickness = 1.5
flierprops = dict(marker='o',markersize=1.5)
boxplot_kwargs = {'widths':0.65, 'vert':True, 'patch_artist':True, 'boxprops':dict(facecolor='#FEFBF3'), 'whis':[5, 95]}
bar_kwargs = {'color':'#C5DEFF', 'edgecolor':'#000000', 'capsize':4, 'linewidth':1.5}
plt.rcParams['axes.linewidth'] = axesthickness
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'Arial'

### Import list of all experiments

In [37]:
path='C:\\Users\\nimath\\switchdrive\\Institution\\PhD\\01_Experiments\\05_Various\\08_Lab\\02_OffTarget\\CjCas9\\AnalysisFiles\\'

def list_files1(directory):
    return [f for f in listdir(directory) if ('Cj' in f) and ('6bpwindow' in f)]

sampledf = pd.DataFrame(columns=['filename','Editor','Replicate'])
filelist = list_files1(path)
sampledf['filename'] = filelist
sampledf['Library'] = 'Cj_Offtarget'
sampledf['Editor'] = sampledf['filename'].apply(lambda x: x.split('_')[3])
sampledf['Replicate'] = sampledf['filename'].apply(lambda x: x.split('_')[-1].split('.')[0])
sampledf['Control'] = sampledf['filename'].apply(lambda x: True if 'Ctrl' in x else False)
sampledf['repname'] = sampledf.apply(lambda x: x.Editor+"_"+x.Replicate,axis=1)

In [38]:
templatedf = pd.read_csv('C:\\Users\\nimath\\switchdrive\\Institution\\PhD\\01_Experiments\\05_Various\\08_Lab\\02_OffTarget\\CjCas9\\CjCas9_OFFLib.csv')

In [39]:
sampledf

,filename,Editor,Replicate,Library,Control,repname
0,20221011_6bpwindow_06-RT_CjCtrl_rep3.csv,CjCtrl,rep3,Cj_Offtarget,True,CjCtrl_rep3
1,20221011_6bpwindow_07-RT_CjWT_rep3.csv,CjWT,rep3,Cj_Offtarget,False,CjWT_rep3
2,20221011_6bpwindow_08-RT_CjEn_rep3.csv,CjEn,rep3,Cj_Offtarget,False,CjEn_rep3
3,20221011_6bpwindow_09-RT_CjEvo_rep3.csv,CjEvo,rep3,Cj_Offtarget,False,CjEvo_rep3
4,20221011_6bpwindow_18-RT_CjCtrl_rep2.csv,CjCtrl,rep2,Cj_Offtarget,True,CjCtrl_rep2
5,20221011_6bpwindow_19-RT_CjWT_rep2.csv,CjWT,rep2,Cj_Offtarget,False,CjWT_rep2
6,20221011_6bpwindow_20-RT_CjEn_rep2.csv,CjEn,rep2,Cj_Offtarget,False,CjEn_rep2
7,20221011_6bpwindow_21-RT_CjEvo_rep2.csv,CjEvo,rep2,Cj_Offtarget,False,CjEvo_rep2
8,20221011_6bpwindow_30-RT_CjCtrl_rep1.csv,CjCtrl,rep1,Cj_Offtarget,True,CjCtrl_rep1
9,20221011_6bpwindow_31-RT_CjWT_rep1.csv,CjWT,rep1,Cj_Offtarget,False,CjWT_rep1


### Calculate control characteristics for each experimental setup

In [40]:
dataframe = pd.read_csv(path+'20221011_6bpwindow_06-RT_CjCtrl_rep3.csv')
dataframe

,Unnamed: 0,Identifier,Name,Number,5'Overhang,spacer,Unnamed: 5,scaffold,filler,random_left_4n,...,Barcode,3'Overhang,Concat:,Length:,spacer_part,Spacer_Length,target,uneditedcount,indelcount,totalreads
0,0,Cj_001,Cj,1,GTGGAAAGGACGAAACACCG,CTGTCACCAATCCTGTCCCTAG,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,AAAC,...,ACTGGCG,GACGTGACTCCTATGACGCTTCT,GTGGAAAGGACGAAACACCGCTGTCACCAATCCTGTCCCTAGGTTC...,203,CTGTCAC,22,TCCCTA,5,2866,2871.0
1,1,Cj_002,Cj,2,GTGGAAAGGACGAAACACCG,AAGGGAACCCAGCGAGTGAAGA,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,GACT,...,AGTACCG,GACGTGACTCCTATGACGCTTCT,GTGGAAAGGACGAAACACCGAAGGGAACCCAGCGAGTGAAGAGTTC...,203,AAGGGAA,22,GTGAAG,511,26,537.0
2,2,Cj_003,Cj,3,GTGGAAAGGACGAAACACCG,TGGGTGAGGGAGGAGAGATGCC,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,AGTT,...,TGCCTTA,GACGTGACTCCTATGACGCTTCT,GTGGAAAGGACGAAACACCGTGGGTGAGGGAGGAGAGATGCCGTTC...,203,TGGGTGA,22,AGATGC,122,47,169.0
3,3,Cj_004,Cj,4,GTGGAAAGGACGAAACACCG,CAGCGCCGAGAAGGAAGTGCTC,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,ATGG,...,TTCAGTA,GACGTGACTCCTATGACGCTTCT,GTGGAAAGGACGAAACACCGCAGCGCCGAGAAGGAAGTGCTCGTTC...,203,CAGCGCC,22,AGTGCT,0,5,5.0
4,4,Cj_005,Cj,5,GTGGAAAGGACGAAACACCG,CTGTCCCCTCCACCCCACAGTG,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,TAAA,...,GGTACCA,GACGTGACTCCTATGACGCTTCT,GTGGAAAGGACGAAACACCGCTGTCCCCTCCACCCCACAGTGGTTC...,203,CTGTCCC,22,CACAGT,300,29,329.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,3117,Cj_3118,Cj,3118,GTGGAAAGGACGAAACACCG,GCCCAGCATGCTGCCCTTGACT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,ACCC,...,TATACCA,CTTGTGACTCCTATGACGCTTCT,GTGGAAAGGACGAAACACCGGCCCAGCATGCTGCCCTTGACTGTTC...,203,GCCCAGC,22,CAAGAC,620,103,723.0
3118,3118,Cj_3119,Cj,3119,GTGGAAAGGACGAAACACCG,GCCCAGCATGCTGCCCTTGACT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,TAAA,...,ATCCATG,CTTGTGACTCCTATGACGCTTCT,GTGGAAAGGACGAAACACCGGCCCAGCATGCTGCCCTTGACTGTTC...,203,GCCCAGC,22,CTACAC,223,57,280.0
3119,3119,Cj_3120,Cj,3120,GTGGAAAGGACGAAACACCG,GCCCAGCATGCTGCCCTTGACT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,TTCG,...,AAGTTCC,CTTGTGACTCCTATGACGCTTCT,GTGGAAAGGACGAAACACCGGCCCAGCATGCTGCCCTTGACTGTTC...,203,GCCCAGC,22,CTTCTC,428,119,547.0
3120,3120,Cj_3121,Cj,3121,GTGGAAAGGACGAAACACCG,GCCCAGCATGCTGCCCTTGACT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,ATCT,...,GGTAAAA,CTTGTGACTCCTATGACGCTTCT,GTGGAAAGGACGAAACACCGGCCCAGCATGCTGCCCTTGACTGTTC...,203,GCCCAGC,22,CTTGTG,158,64,222.0


In [41]:
ctrllist = sampledf[sampledf['Editor'] == 'CjCtrl']
ctrldf = templatedf.copy()
for index, row in ctrllist.iterrows():
    controlrepdf = pd.read_csv(path+row.filename)
    ctrldf[row.repname+'_totalreads'] = controlrepdf['totalreads']
    controlrepdf['percentageindel'] = controlrepdf.apply(lambda x: (x.indelcount/x.totalreads)*100 ,axis=1)
    controlrepdf['percentageunedited'] = controlrepdf.apply(lambda x: (x.uneditedcount/x.totalreads)*100 ,axis=1)
    ctrldf[row.repname+'_percentageunedited'] = controlrepdf['percentageunedited']
    ctrldf[row.repname+'_percentageindel'] = controlrepdf['percentageindel']
    ctrldf[row.repname+'_uneditedcount'] = controlrepdf['uneditedcount']
    ctrldf[row.repname+'_indelcount'] = controlrepdf['indelcount']

for index, row in ctrllist.iterrows():    
    ctrldf = ctrldf[ctrldf[row.repname+'_totalreads'] > 50]
    ctrldf = ctrldf[ctrldf[row.repname+'_percentageindel'] < 60]

In [42]:
ctrldf

,Identifier,Name,Number,5'Overhang,spacer,Unnamed: 5,scaffold,filler,random_left_4n,PAM_rv,...,CjCtrl_rep2_totalreads,CjCtrl_rep2_percentageunedited,CjCtrl_rep2_percentageindel,CjCtrl_rep2_uneditedcount,CjCtrl_rep2_indelcount,CjCtrl_rep1_totalreads,CjCtrl_rep1_percentageunedited,CjCtrl_rep1_percentageindel,CjCtrl_rep1_uneditedcount,CjCtrl_rep1_indelcount
1,Cj_002,Cj,2,GTGGAAAGGACGAAACACCG,AAGGGAACCCAGCGAGTGAAGA,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,GACT,GTATTCCGT,...,836.0,95.933014,4.066986,802,34,636.0,93.710692,6.289308,596,40
2,Cj_003,Cj,3,GTGGAAAGGACGAAACACCG,TGGGTGAGGGAGGAGAGATGCC,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,AGTT,GTATTCCGG,...,301.0,77.740864,22.259136,234,67,270.0,78.518519,21.481481,212,58
4,Cj_005,Cj,5,GTGGAAAGGACGAAACACCG,CTGTCCCCTCCACCCCACAGTG,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,TAAA,GTATTCCCC,...,558.0,92.114695,7.885305,514,44,401.0,88.029925,11.970075,353,48
5,Cj_006,Cj,6,GTGGAAAGGACGAAACACCG,GTGGGGCCACTAGGGACAGGAT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,TAGT,GTATTCCAA,...,131.0,45.038168,54.961832,59,72,73.0,56.164384,43.835616,41,32
6,Cj_007,Cj,7,GTGGAAAGGACGAAACACCG,CTGTCCCTAGTGGCCCCACTGT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,GAGC,GTATTCCCA,...,448.0,89.955357,10.044643,403,45,316.0,93.987342,6.012658,297,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,Cj_3118,Cj,3118,GTGGAAAGGACGAAACACCG,GCCCAGCATGCTGCCCTTGACT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,ACCC,GTGTTGCTA,...,666.0,85.135135,14.864865,567,99,690.0,86.231884,13.768116,595,95
3118,Cj_3119,Cj,3119,GTGGAAAGGACGAAACACCG,GCCCAGCATGCTGCCCTTGACT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,TAAA,GTGTTGCTA,...,289.0,80.622837,19.377163,233,56,284.0,76.760563,23.239437,218,66
3119,Cj_3120,Cj,3120,GTGGAAAGGACGAAACACCG,GCCCAGCATGCTGCCCTTGACT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,TTCG,GTGTTGCTA,...,536.0,70.708955,29.291045,379,157,525.0,72.571429,27.428571,381,144
3120,Cj_3121,Cj,3121,GTGGAAAGGACGAAACACCG,GCCCAGCATGCTGCCCTTGACT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,ATCT,GTGTTGCTA,...,186.0,75.806452,24.193548,141,45,144.0,69.444444,30.555556,100,44


In [43]:
editorlist = sampledf['Editor'].unique()
editorlist = [x for x in editorlist if not 'Ctrl' in x]
experimentdict = {}
for editor in editorlist:
    experimentdf = templatedf.copy()
    for index, row in sampledf[sampledf['Editor'] == editor].iterrows():
        dataframe = pd.read_csv(path+row.filename)
        dataframename = row.Editor+"_"+row.Replicate
        experimentdf[dataframename+'_totalreads'] = dataframe['totalreads']
        dataframe['percentageindel'] = dataframe.apply(lambda x: (x.indelcount/x.totalreads)*100 ,axis=1)
        dataframe['percentageunedited'] = dataframe.apply(lambda x: (x.uneditedcount/x.totalreads)*100 ,axis=1)
        experimentdf[dataframename+'_percentageunedited'] = dataframe['percentageunedited']
        experimentdf[dataframename+'_percentageindel'] = dataframe['percentageindel']
        experimentdf[dataframename+'_uneditedcount'] = dataframe['uneditedcount']
        experimentdf[dataframename+'_indelcount'] = dataframe['indelcount']
    replicates = list(sampledf[sampledf['Editor'] == editor].repname)
    controldf = sampledf[(sampledf['Editor'] == 'CjCtrl')]
    #controlname = controldf['Cell_line_Editor'] + '_' + controldf['Replicate']
    
    ctrluneditedlist = []
    ctrlindellist = []
    lib = "CjOfftarget"
    for index, row in ctrllist.iterrows():
        ctrluneditedlist.append(row.repname+'_percentageunedited')
        ctrlindellist.append(row.repname+'_percentageindel')
        
    experimentdf[lib+'_ctr'+'_percentageunedited'] = ctrldf[ctrluneditedlist].mean(axis=1)
    experimentdf[lib+'_ctr'+'_percentageindel'] = ctrldf[ctrlindellist].mean(axis=1)
    #print()
    #print(controlname)
    #print(cellline)
    cutoff = 50
    initialtemplen = len(experimentdf)
    #print('Total variants:',initialtemplen)
    for replicate in replicates:
        experimentdf = experimentdf[experimentdf[replicate+'_totalreads']>cutoff]

    experimentdict[editor] = experimentdf

### Calculate editing characteristics for all experimental setups
Also clip editing levels to be between 0-100.

In [44]:
sampledf_woctr = sampledf[sampledf['Editor'] != 'CjCtrl']

In [45]:
editordf = sampledf_woctr.copy()
editordf = editordf[editordf['Replicate'] == "rep1"]
editordf = editordf.set_index('Editor')

cols = ['Library']
editordf = editordf[cols]
editordf

,Library
Editor,
CjWT,Cj_Offtarget
CjEn,Cj_Offtarget
CjEvo,Cj_Offtarget


In [46]:
for editor in experimentdict:
    print(editor)

CjWT
CjEn
CjEvo


In [47]:
for editor in experimentdict:
    print(editor)
    replicatelist = list(sampledf_woctr[sampledf_woctr['Editor'] == editor]['repname'])
    ctrname = lib+'_ctr'
    
    for replicate in replicatelist:
        print(replicate)
        repnr = replicate.split('_')[-1]
        experimentdict[editor][replicate+'_percentageindel_corrected_individual'] = experimentdict[editor].apply(lambda x: ((x[replicate+'_percentageindel']-x[ctrname+'_percentageindel'])/((100-x[ctrname+'_percentageindel'])/100)),axis=1)
        editordf.at[editor,repnr +'_meanindel'] = None
        editordf.at[editor,repnr +'_meanindel'] = experimentdict[editor][[replicate +'_percentageindel_corrected_individual']].mean()
        
    allreps = []
    for index,row in sampledf_woctr[sampledf_woctr['Editor'] == editor].iterrows():
        rep = row.Editor+'_'+row.Replicate
        allreps.append(rep)
        
    print(allreps)    
    experimentdict[editor][editor+'_averageindel'] = experimentdict[editor][[x +'_percentageindel_corrected_individual' for x in allreps]].mean(axis=1)
    
       
    # Clip editing rates to be between 0 and 100
    experimentdict[editor][editor+'_averageindel'] = experimentdict[editor][editor+'_averageindel'].clip(0,100)
    
    experimentdict[editor] = experimentdict[editor].dropna(subset=[editor+'_averageindel'])
    
    replist = ['1_meanindel','2_meanindel','3_meanindel']
    
    
    
    
    editordf.at[editor,'numberofvariants'] = len(experimentdict[editor])
    editordf.at[editor,'meanindel'] = experimentdict[editor][editor+'_averageindel'].mean()
    editordf.at[editor,'standarddeviation'] = experimentdict[editor][editor+'_averageindel'].std()
    experimentdict[editor].to_csv('.//AnalysisFiles//summary//20221012_'+editor+'_analysis_6bpwindow_dataframe.csv')

CjWT
CjWT_rep3
CjWT_rep2
CjWT_rep1
['CjWT_rep3', 'CjWT_rep2', 'CjWT_rep1']
CjEn
CjEn_rep3
CjEn_rep2
CjEn_rep1
['CjEn_rep3', 'CjEn_rep2', 'CjEn_rep1']
CjEvo
CjEvo_rep3
CjEvo_rep2
CjEvo_rep1
['CjEvo_rep3', 'CjEvo_rep2', 'CjEvo_rep1']


In [48]:
editordf

,Library,rep3_meanindel,rep2_meanindel,rep1_meanindel,numberofvariants,meanindel,standarddeviation
Editor,,,,,,,
CjWT,Cj_Offtarget,1.991858,1.517704,2.122555,2626.0,2.238417,4.119281
CjEn,Cj_Offtarget,3.695937,4.844242,3.836683,2690.0,4.653770,9.286083
CjEvo,Cj_Offtarget,3.073427,3.160065,2.721962,2825.0,3.303598,5.915402


In [49]:
sampledf_woctr

,filename,Editor,Replicate,Library,Control,repname
1,20221011_6bpwindow_07-RT_CjWT_rep3.csv,CjWT,rep3,Cj_Offtarget,False,CjWT_rep3
2,20221011_6bpwindow_08-RT_CjEn_rep3.csv,CjEn,rep3,Cj_Offtarget,False,CjEn_rep3
3,20221011_6bpwindow_09-RT_CjEvo_rep3.csv,CjEvo,rep3,Cj_Offtarget,False,CjEvo_rep3
5,20221011_6bpwindow_19-RT_CjWT_rep2.csv,CjWT,rep2,Cj_Offtarget,False,CjWT_rep2
6,20221011_6bpwindow_20-RT_CjEn_rep2.csv,CjEn,rep2,Cj_Offtarget,False,CjEn_rep2
7,20221011_6bpwindow_21-RT_CjEvo_rep2.csv,CjEvo,rep2,Cj_Offtarget,False,CjEvo_rep2
9,20221011_6bpwindow_31-RT_CjWT_rep1.csv,CjWT,rep1,Cj_Offtarget,False,CjWT_rep1
10,20221011_6bpwindow_32-RT_CjEn_rep1.csv,CjEn,rep1,Cj_Offtarget,False,CjEn_rep1
11,20221011_6bpwindow_33-RT_CjEvo_rep1.csv,CjEvo,rep1,Cj_Offtarget,False,CjEvo_rep1


In [50]:
summarydf = templatedf.copy()
for editor in experimentdict:
    summarydf[editor+'_averageindel'] = experimentdict[editor][editor+'_averageindel']
    summarydf[editor+'_rep1_percentageindel_corrected_individual'] = experimentdict[editor][editor+'_rep1_percentageindel_corrected_individual']
    summarydf[editor+'_rep2_percentageindel_corrected_individual'] = experimentdict[editor][editor+'_rep2_percentageindel_corrected_individual']
    summarydf[editor+'_rep3_percentageindel_corrected_individual'] = experimentdict[editor][editor+'_rep3_percentageindel_corrected_individual']
    
    

In [51]:
summarydf = summarydf.dropna(subset=['CjWT_averageindel','CjEn_averageindel','CjEvo_averageindel'])
summarydf.to_csv('AnalysisFiles/summary/20221012_CjCas_alleditors_analysis_6bpwindow_dataframe.csv')

In [52]:
experimentdict[editor]

,Identifier,Name,Number,5'Overhang,spacer,Unnamed: 5,scaffold,filler,random_left_4n,PAM_rv,...,CjEvo_rep1_percentageunedited,CjEvo_rep1_percentageindel,CjEvo_rep1_uneditedcount,CjEvo_rep1_indelcount,CjOfftarget_ctr_percentageunedited,CjOfftarget_ctr_percentageindel,CjEvo_rep3_percentageindel_corrected_individual,CjEvo_rep2_percentageindel_corrected_individual,CjEvo_rep1_percentageindel_corrected_individual,CjEvo_averageindel
1,Cj_002,Cj,2,GTGGAAAGGACGAAACACCG,AAGGGAACCCAGCGAGTGAAGA,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,GACT,GTATTCCGT,...,87.417219,12.582781,396,57,94.933998,5.066002,14.360698,-1.339841,7.917900,6.979586
2,Cj_003,Cj,3,GTGGAAAGGACGAAACACCG,TGGGTGAGGGAGGAGAGATGCC,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,AGTT,GTATTCCGG,...,41.139241,58.860759,130,186,76.149577,23.850423,9.234365,30.437465,45.975747,28.549192
4,Cj_005,Cj,5,GTGGAAAGGACGAAACACCG,CTGTCCCCTCCACCCCACAGTG,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,TAAA,GTATTCCCC,...,76.940639,23.059361,337,101,90.443344,9.556656,7.591085,-2.127920,14.929462,6.797542
7,Cj_008,Cj,8,GTGGAAAGGACGAAACACCG,CAGAGCCACATTAACCGGCCCT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,CGGC,GTATTCCCA,...,95.658073,4.341927,705,32,93.868231,6.131769,20.223496,12.599046,-1.906761,10.305261
8,Cj_009,Cj,9,GTGGAAAGGACGAAACACCG,CAGCTCGGGGACACAGGATCCC,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,CTTT,GTATTCCAG,...,93.037975,6.962025,147,11,85.777503,14.222497,1.127774,2.849430,-8.464307,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,Cj_3118,Cj,3118,GTGGAAAGGACGAAACACCG,GCCCAGCATGCTGCCCTTGACT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,ACCC,GTGTTGCTA,...,81.674208,18.325792,361,81,85.706941,14.293059,10.578514,11.469022,4.705258,8.917598
3118,Cj_3119,Cj,3119,GTGGAAAGGACGAAACACCG,GCCCAGCATGCTGCCCTTGACT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,TAAA,GTGTTGCTA,...,81.026439,18.973561,521,122,79.008753,20.991247,-1.706634,-3.773973,-2.553750,0.000000
3119,Cj_3120,Cj,3120,GTGGAAAGGACGAAACACCG,GCCCAGCATGCTGCCCTTGACT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,TTCG,GTGTTGCTA,...,74.280230,25.719770,387,134,73.841785,26.158215,2.193288,12.607720,-0.593763,4.735749
3120,Cj_3121,Cj,3121,GTGGAAAGGACGAAACACCG,GCCCAGCATGCTGCCCTTGACT,22,GTTCTAGTCCCTGAAAAGGGACTAGAATAAAGAGTTTGCGGGACTC...,GTGC,ATCT,GTGTTGCTA,...,78.225806,21.774194,97,27,72.140689,27.859311,-0.498070,25.810070,-8.435070,5.625644
